In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np


# ---------- 1. Чтение молекул из SDF и генерация 3D-координат ----------

def load_molecules_with_3d(sdf_path: str):
    """
    Загружает молекулы из SDF и гарантирует, что у каждой есть 3D-конформер.
    Возвращает список RDKit-молекул.
    """
    suppl = Chem.SDMolSupplier(sdf_path, removeHs=False)
    mols = []

    for mol in suppl:
        if mol is None:
            continue

        # Если нет 3D-конформера — генерируем
        if mol.GetNumConformers() == 0:
            mol = Chem.AddHs(mol)
            params = AllChem.ETKDGv3()
            params.randomSeed = 0
            AllChem.EmbedMolecule(mol, params)
            AllChem.UFFOptimizeMolecule(mol)

        mols.append(mol)

    return mols


def mol_to_coords(mol) -> np.ndarray:
    """
    Достает 3D-координаты атомов из молекулы.
    Возвращает массив shape (n_atoms, 3).
    """
    conf = mol.GetConformer()
    coords = []
    for atom in mol.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        coords.append([pos.x, pos.y, pos.z])
    return np.array(coords, dtype=float)


# ---------- 2. Нормализация в единичный шар ----------

def normalize_coords_to_unit_sphere(coords: np.ndarray) -> np.ndarray:
    """
    1) Сдвиг в центр масс.
    2) Масштабирование так, чтобы max расстояние до центра было = 1.
       То есть вся молекула помещается в шар радиуса 1.
    """
    center = coords.mean(axis=0)
    coords_centered = coords - center

    radii = np.linalg.norm(coords_centered, axis=1)
    max_r = radii.max()

    if max_r == 0:
        # Вырожденный случай: все атомы в одной точке
        return coords_centered

    coords_norm = coords_centered / max_r
    return coords_norm  # теперь max ||r|| ≈ 1


# ---------- 3. Преобразование координат в воксельный куб ----------

def coords_to_voxel_grid(coords: np.ndarray,
                         grid_size: int = 32) -> np.ndarray:
    """
    Превращает координаты в [-1,1]^3 в воксельный куб размера grid_size^3.
    Простая модель: каждый атом -> один воксель +1.

    На выходе: np.array shape (grid_size, grid_size, grid_size).
    """
    grid = np.zeros((grid_size, grid_size, grid_size), dtype=np.float32)

    # Функция перевода координат [-1,1] -> индексы [0, grid_size-1]
    def to_index(x):
        # x в [-1,1] -> [0, grid_size-1]
        idx = (x + 1) * 0.5 * (grid_size - 1)
        idx = np.round(idx).astype(int)
        idx = np.clip(idx, 0, grid_size - 1)
        return idx

    idxs = to_index(coords)  # shape (n_atoms, 3)

    for ix, iy, iz in idxs:
        grid[ix, iy, iz] += 1.0

    # (опционально) нормируем так, чтобы максимум был 1.0
    max_val = grid.max()
    if max_val > 0:
        grid /= max_val

    return grid


# ---------- 4. Полный конвейер: SDF -> npy с объёмами ----------

def sdf_to_voxel_volumes(sdf_path: str,
                         grid_size: int = 32) -> np.ndarray:
    """
    Основная функция:
    - читает молекулы из SDF,
    - превращает каждую в нормализованный 3D-воксельный объем
      внутри единичного шара.

    Возвращает массив shape (n_mols, grid_size, grid_size, grid_size).
    """
    mols = load_molecules_with_3d(sdf_path)
    volumes = []

    for mol in mols:
        coords = mol_to_coords(mol)
        coords_unit = normalize_coords_to_unit_sphere(coords)
        grid = coords_to_voxel_grid(coords_unit, grid_size=grid_size)
        volumes.append(grid)

    return np.stack(volumes)


if __name__ == "__main__":
    # 1. Путь к твоему файлу
    sdf_path = "100_time_steps_generated_samples.sdf"  # <- сюда твой путь
    grid_size = 32

    # 2. Считаем воксельные представления для всех молекул
    volumes = sdf_to_voxel_volumes(sdf_path, grid_size=grid_size)

    print("Форма массива с молекулами:", volumes.shape)
    # (n_mols, 32, 32, 32), например (100, 32, 32, 32)

    # 3. Сохраняем в файл, чтобы потом использовать для Цернике
    np.save("molecule_volumes.npy", volumes)

    # Пример: берём первую молекулу
    first_vol = volumes[0]
    print("Максимум в первом объёме:", first_vol.max())


Форма массива с молекулами: (5045, 32, 32, 32)
Максимум в первом объёме: 1.0


In [3]:
import numpy as np
import ZMPY3D  # пакет уже установлен

# 1. Берём объём одной молекулы
volumes = np.load("molecule_volumes.npy")  # (5045, 32, 32, 32)
vol = volumes[0].astype(np.float32)        # (32, 32, 32)

# 2. Импортируем модуль/класс из ZMPY3D
# Пример: если в dir(ZMPY3D) ты увидишь 'ZMPY3D_numpy',
# можно сделать так (ЕСЛИ название именно такое):
import ZMPY3D.ZMPY3D_numpy as zm  # <--- ЗАМЕНИ на реальное имя

# 3. Задаём максимальный порядок моментов
max_order = 10  # для теста, выше = дольше, но точнее

# 4. Вычисляем 3D Zernike
# Здесь нужно заменить 'compute_3d_zernike' на реальную функцию/метод
# из zm, который принимает объем и порядок.
# Возможные варианты (пример — ты выберешь правильный по help(zm)):

# Вариант А: если есть функция
coeffs = zm.compute_3d_zernike(vol, max_order=max_order)

# Вариант Б: если есть класс ZernikeMoments
# zm_obj = zm.ZernikeMoments(max_order=max_order, cube_size=vol.shape[0])
# coeffs = zm_obj.compute(vol)

print(type(coeffs))
print("Размер вектора моментов:", np.array(coeffs).shape)

# 5. Делаем инвариантный вектор признаков (модули комплексных коэффициентов)
coeffs = np.asarray(coeffs)
features = np.abs(coeffs)

print("Признаки для одной молекулы, shape:", features.shape)


ModuleNotFoundError: No module named 'ZMPY3D.ZMPY3D_numpy'